In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
import joblib
import numpy as np
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif

In [8]:
# Show all columns in the DataFrame
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [9]:
# Load raw data
#benign = pd.read_csv('datasets/hmi_normal_6h_flow_label.csv')
scapy_modify = pd.read_csv('datasets/mitm_hmi_scapy_modify_1plc_6h_flow_label.csv')
scapy_modify_12h = pd.read_csv('datasets/mitm_hmi_scapy_modify_1plc_12h_flow_label.csv')

In [10]:
label_counts = scapy_modify['Label'].value_counts()
label_percent = scapy_modify['Label'].value_counts(normalize=True) * 100
result = pd.DataFrame({'Count': label_counts, 'Percentage (%)': label_percent.round(2)})
print(result)

         Count  Percentage (%)
Label                         
normal  164457           96.06
mitm      6754            3.94


In [11]:
label_counts = scapy_modify_12h['Label'].value_counts()
label_percent = scapy_modify_12h['Label'].value_counts(normalize=True) * 100
result = pd.DataFrame({'Count': label_counts, 'Percentage (%)': label_percent.round(2)})
print(result)

         Count  Percentage (%)
Label                         
Benign  341297           96.05
MITM     14024            3.95


In [12]:
# Step 2: Filter only the MITM rows from scapy_modify_12h
mitm_12h = scapy_modify_12h[scapy_modify_12h["Label"] == "MITM"].copy()

# Step 3: Rename "MITM" label to "mitm"
mitm_12h["Label"] = "mitm"

In [13]:
# Drop unnecessary columns
cols_to_drop = ['Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Timestamp']
#benign.drop(columns=cols_to_drop, errors='ignore', inplace=True)
scapy_modify.drop(columns=cols_to_drop, errors='ignore', inplace=True)
mitm_12h.drop(columns=cols_to_drop,errors='ignore', inplace=True)

In [14]:
# Merge
df = pd.concat([scapy_modify,mitm_12h], ignore_index=True)

In [15]:
df.groupby("Label").describe()

Protocol                                    Flow Duration  \
           count mean  std  min  25%  50%  75%  max         count   
Label                                                               
mitm     20778.0  6.0  0.0  6.0  6.0  6.0  6.0  6.0       20778.0   
normal  164457.0  6.0  0.0  6.0  6.0  6.0  6.0  6.0      164457.0   

                                                                      \
                 mean           std   min   25%       50%        75%   
Label                                                                  
mitm    473679.333959  2.080583e+06  13.0  38.0  618029.5  833067.25   
normal   58148.334805  1.424615e+06   7.0  14.0   57859.0   73380.00   

                    Tot Fwd Pkts                                               \
                max        count      mean       std  min  25%  50%  75%  max   
Label                                                                           
mitm    108012241.0      20778.0  2.412215  2.425780  0.0  0.0  4.0  5.0  6.0   
normal  117511126.0     164457.0  2.459804  2.466897  0.0  0.0  4.0  5.0  6.0   

       Tot Bwd Pkts                                               \
              count      mean       std  min  25%  50%  75%  max   
Label                                                              
mitm        20778.0  3.590577  1.615819  1.0  2.0  5.0  5.0  8.0   
normal     164457.0  3.541515  1.553819  2.0  2.0  5.0  5.0  8.0   

       TotLen Fwd Pkts                                                     \
                 count       mean        std  min  25%   50%   75%    max   
Label                                                                       
mitm           20778.0  46.895948  47.389270  0.0  0.0  72.0  98.0  168.0   
normal        164457.0  47.981515  48.193465  0.0  0.0  72.0  96.0  100.0   

       TotLen Bwd Pkts                                                     \
                 count       mean        std  min  25%   50%   75%    max   
Label                                                                       
mitm           20778.0  39.175667  39.753687  0.0  0.0  72.0  72.0  168.0   
normal        164457.0  38.461665  38.756286  0.0  0.0  72.0  78.0  100.0   

       Fwd Pkt Len Max                                                    \
                 count       mean        std  min  25%   50%   75%   max   
Label                                                                      
mitm           20778.0  32.848494  33.508172  0.0  0.0  44.0  70.0  72.0   
normal        164457.0  33.978535  34.284204  0.0  0.0  44.0  68.0  72.0   

       Fwd Pkt Len Min                                    Fwd Pkt Len Mean  \
                 count mean  std  min  25%  50%  75%  max            count   
Label                                                                        
mitm           20778.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0          20778.0   
normal        164457.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0         164457.0   

                                                       Fwd Pkt Len Std  \
            mean       std  min  25%   50%   75%   max           count   
Label                                                                    
mitm    9.707837  9.716730  0.0  0.0  18.0  19.6  28.0         20778.0   
normal  9.749339  9.753269  0.0  0.0  18.0  19.2  20.0        164457.0   

                                                                         \
             mean        std  min  25%        50%        75%        max   
Label                                                                     
mitm    14.616392  14.786931  0.0  0.0  21.786846  30.672463  34.292856   
normal  14.990621  15.069873  0.0  0.0  21.786846  29.852973  31.496031   

       Bwd Pkt Len Max                                                    \
                 count       mean        std  min  25%   50%   75%   max   
Label                                                                      
mitm           20778.0  25.168255  

In [16]:
label_counts = df['Label'].value_counts()
label_percent = df['Label'].value_counts(normalize=True) * 100
result = pd.DataFrame({'Count': label_counts, 'Percentage (%)': label_percent.round(2)})
print(result)

         Count  Percentage (%)
Label                         
normal  164457           88.78
mitm     20778           11.22


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

for col in df.select_dtypes(include='number').columns:
    plt.figure(figsize=(12, 4))
    sns.boxplot(y='Label', x=col, data=df)
    plt.title(f'Boxplot of {col} by Label')
    plt.show()


In [ ]:
for col in df.select_dtypes(include='number').columns:
    plt.figure(figsize=(12, 4))
    sns.violinplot(y='Label', x=col, data=df)
    plt.title(f'Violin plot of {col} by Label')
    plt.show()

In [ ]:
from scipy.stats import ttest_ind

label_values = df['Label'].unique()
group1 = df[df['Label'] == label_values[0]]
group2 = df[df['Label'] == label_values[1]]

for col in df.select_dtypes(include='number').columns:
    stat, p = ttest_ind(group1[col], group2[col], nan_policy='omit')
    print(f'{col}: t={stat:.2f}, p={p:.4f}')



In [ ]:
# from sklearn.preprocessing import LabelEncoder

# # Create and fit the encoder
# le = LabelEncoder()
# df['Label_encoded'] = le.fit_transform(df['Label'])

# # Create mapping dictionary
# label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))

# print("Label to Integer Mapping:")
# for label, code in label_mapping.items():
#     print(f"'{label}' → {code}")


In [ ]:
# reverse_mapping = dict(zip(le.transform(le.classes_), le.classes_))

# print("\nInteger to Label Mapping:")
# for code, label in reverse_mapping.items():
#     print(f"{code} → '{label}'")


In [ ]:
numeric_df = df.select_dtypes(include='number')
constant_cols = [col for col in numeric_df.columns if numeric_df[col].nunique() == 1]
df = df.drop(columns=constant_cols)


In [ ]:
for col in df.select_dtypes(include='number').columns:
    plt.figure(figsize=(12, 4))
    sns.violinplot(y='Label', x=col, data=df)
    plt.title(f'Violin plot of {col} by Label')
    plt.show()

In [ ]:
from scipy.stats import f_oneway

# Assume 'Label' column exists and has at least 2 unique values
label_values = df['Label'].unique()

# Store non-informative columns
uninformative_cols = []

# Loop through numeric columns
for col in df.select_dtypes(include='number').columns:
    if col == 'Label':
        continue
    # Group data by label
    groups = [df[df['Label'] == label][col].dropna() for label in label_values]
    
    # Check if all groups are constant or empty
    if all(g.nunique() <= 1 for g in groups):
        uninformative_cols.append(col)
        continue

    # Use ANOVA test to see if means differ significantly
    try:
        stat, p = f_oneway(*groups)
        if p > 0.05:  # Not significantly different
            uninformative_cols.append(col)
    except:
        pass  # Skip if group is too small or identical

# Drop them
df = df.drop(columns=uninformative_cols)

# Optional: print dropped columns
print("Dropped columns that do not differ across labels:", uninformative_cols)


In [ ]:
for col in df.select_dtypes(include='number').columns:
    plt.figure(figsize=(12, 4))
    sns.violinplot(y='Label', x=col, data=df)
    plt.title(f'Violin plot of {col} by Label')
    plt.show()

In [ ]:
label_values = df['Label'].unique()
label_values

In [ ]:
from scipy.stats import ks_2samp


uninformative_cols = []

# Only valid for binary labels

label_1 = 'normal'
label_2 = 'mitm_36plc'

for col in df.select_dtypes(include='number').columns:
    if col == 'Label':
        continue
    group1 = df[df['Label'] == label_1][col].dropna()
    group2 = df[df['Label'] == label_2][col].dropna()

    if group1.nunique() <= 1 and group2.nunique() <= 1:
        uninformative_cols.append(col)
        continue

    stat, p = ks_2samp(group1, group2)
    if p > 0.05:  # Not significantly different distributions
        uninformative_cols.append(col)

df = df.drop(columns=uninformative_cols)
print("Dropped (KS test):", uninformative_cols)


In [ ]:
from scipy.stats import kruskal

label_values = df['Label'].unique()
uninformative_cols = []

for col in df.select_dtypes(include='number').columns:
    if col == 'Label':
        continue
    groups = [df[df['Label'] == lv][col].dropna() for lv in label_values]

    if all(g.nunique() <= 1 for g in groups):
        uninformative_cols.append(col)
        continue

    try:
        stat, p = kruskal(*groups)
        if p > 0.05:
            uninformative_cols.append(col)
    except:
        pass

df = df.drop(columns=uninformative_cols)
print("Dropped (Kruskal):", uninformative_cols)


In [ ]:
for col in df.select_dtypes(include='number').columns:
    plt.figure(figsize=(12, 4))
    sns.violinplot(y='Label', x=col, data=df)
    plt.title(f'Violin plot of {col} by Label')
    plt.show()

In [ ]:
len(df.columns)

In [ ]:
print(df.columns)